### Merge pheno with peidg files 

- Build a scrubbing file to be used on subtype analysis for wm task
- Add mean run1 and run1 for FD an FD_scrubbed
- Visualize correlation between run 1 and 2 for FD and FD scrubbed
- merge clustered phenotype with scrubbing

In [1]:
# Import pandas
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display

In [2]:
%matplotlib inline

In [3]:
# Set path
#path_root = "~/Drive/HCP/"
path_root = "~/data/data_disk/Drive/HCP2/"
path_pheno = os.path.join(path_root,"subtypes_scores/26-10-2016/pheno/pheno_yu_predic")
predic_file = os.path.join(path_pheno,"rsfmri_REST1_LR_1200R_firstorder__March2019.csv")
unrestric_file = os.path.join(path_pheno,"unrestricted_yuzhang_4_16_2019_10_51_46.csv")
restric_file = os.path.join(path_pheno,"RESTRICTED_yuzhang_4_16_2019_10_53_20.csv")

In [4]:
# Merge restricted and un restricted files 
pheno_pedig_raw = pd.merge(pd.read_csv(restric_file),pd.read_csv(unrestric_file),how='left',on='Subject')
pheno_pedig_raw.head()

,Subject,Age_in_Yrs,HasGT,ZygositySR,ZygosityGT,Family_ID,Mother_ID,Father_ID,TestRetestInterval,Race,...,Noise_Comp,Odor_Unadj,Odor_AgeAdj,PainIntens_RawScore,PainInterf_Tscore,Taste_Unadj,Taste_AgeAdj,Mars_Log_Score,Mars_Errs,Mars_Final
0,100004,24,True,NotTwin,,52259_82122,52259,82122,NaN,White,...,5.2,101.12,86.45,2.0,45.9,107.17,105.31,1.80,0.0,1.80
1,100206,27,True,NotTwin,,56037_85858,56037,85858,NaN,White,...,6.0,108.79,97.19,1.0,49.7,72.63,72.03,1.84,0.0,1.84
2,100307,27,True,NotMZ,MZ,51488_81352,51488,81352,NaN,White,...,3.6,101.12,86.45,0.0,38.6,71.69,71.76,1.76,0.0,1.76
3,100408,33,True,MZ,MZ,51730_81594,51730,81594,NaN,White,...,2.0,108.79,98.04,2.0,52.6,114.01,113.59,1.76,2.0,1.68
4,100610,27,True,NotMZ,DZ,52813_82634,52813,82634,NaN,White,...,2.0,122.25,110.45,0.0,38.6,84.84,85.31,1.92,1.0,1.88


In [5]:
merged_pheno = pd.merge(pheno_pedig_raw[['Subject','Age_in_Yrs','Gender','ZygositySR','Mother_ID','BMI']],
                        pd.read_csv(predic_file,sep='\t'), left_on='Subject',right_on ='subj', how='right')
merged_pheno.drop(columns='subj',inplace= True)
merged_pheno.head()

,Subject,Age_in_Yrs,Gender,ZygositySR,Mother_ID,BMI,fear_emo,non_emo,math_lang,story_lang,...,mental_soc,random_soc,body0b_wm,body2b_wm,face0b_wm,face2b_wm,place0b_wm,place2b_wm,tool0b_wm,tool2b_wm
0,100206,27,M,NotTwin,56037,26.64,0.072917,0.083333,0.437500,0.854167,...,0.364583,0.760417,0.041667,0.156250,0.239583,0.145833,0.145833,0.041667,0.072917,0.145833
1,100307,27,F,NotMZ,51488,22.96,0.106796,0.087379,0.281553,0.533981,...,0.407767,0.883495,0.067961,0.029126,0.097087,0.077670,0.106796,0.048544,0.029126,0.106796
2,100408,33,M,MZ,51730,27.75,0.024590,0.065574,0.401639,0.647541,...,0.196721,0.622951,0.049180,0.040984,0.040984,0.032787,0.008197,0.008197,0.040984,0.040984
3,100610,27,M,NotMZ,52813,36.91,0.016260,0.040650,0.357724,0.487805,...,0.227642,0.756098,0.048780,0.024390,0.032520,0.056911,0.089431,0.032520,0.032520,0.008130
4,101006,35,F,MZ,51283,30.27,0.078431,0.107843,0.470588,0.656863,...,0.333333,1.000000,0.019608,0.058824,0.127451,0.127451,0.058824,0.029412,0.009804,0.078431


In [6]:
# remove rows with missing values
missing = np.array([merged_pheno[col].isnull().values for col in merged_pheno.columns.get_values()]).any(0)
merged_pheno_clean = merged_pheno[~missing]
merged_pheno_clean.count()

Subject        1079
Age_in_Yrs     1079
Gender         1079
ZygositySR     1079
Mother_ID      1079
BMI            1079
fear_emo       1079
non_emo        1079
math_lang      1079
story_lang     1079
foot_mot       1079
hand_mot       1079
tongue_mot     1079
match_reson    1079
relat_reson    1079
mental_soc     1079
random_soc     1079
body0b_wm      1079
body2b_wm      1079
face0b_wm      1079
face2b_wm      1079
place0b_wm     1079
place2b_wm     1079
tool0b_wm      1079
tool2b_wm      1079
dtype: int64

In [7]:
# set gender to numerical
merged_pheno_clean.loc[:,'Gender']= merged_pheno_clean.Gender.replace({'M':1,'F':2})

/home/yassinebha/jupyter_py3.0/lib/python3.5/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [8]:
merged_pheno_clean.head()

,Subject,Age_in_Yrs,Gender,ZygositySR,Mother_ID,BMI,fear_emo,non_emo,math_lang,story_lang,...,mental_soc,random_soc,body0b_wm,body2b_wm,face0b_wm,face2b_wm,place0b_wm,place2b_wm,tool0b_wm,tool2b_wm
0,100206,27,1,NotTwin,56037,26.64,0.072917,0.083333,0.437500,0.854167,...,0.364583,0.760417,0.041667,0.156250,0.239583,0.145833,0.145833,0.041667,0.072917,0.145833
1,100307,27,2,NotMZ,51488,22.96,0.106796,0.087379,0.281553,0.533981,...,0.407767,0.883495,0.067961,0.029126,0.097087,0.077670,0.106796,0.048544,0.029126,0.106796
2,100408,33,1,MZ,51730,27.75,0.024590,0.065574,0.401639,0.647541,...,0.196721,0.622951,0.049180,0.040984,0.040984,0.032787,0.008197,0.008197,0.040984,0.040984
3,100610,27,1,NotMZ,52813,36.91,0.016260,0.040650,0.357724,0.487805,...,0.227642,0.756098,0.048780,0.024390,0.032520,0.056911,0.089431,0.032520,0.032520,0.008130
4,101006,35,2,MZ,51283,30.27,0.078431,0.107843,0.470588,0.656863,...,0.333333,1.000000,0.019608,0.058824,0.127451,0.127451,0.058824,0.029412,0.009804,0.078431


In [9]:
##save to csv
# for solar
merged_pheno_clean.to_csv(os.path.join(path_pheno,"predic_pedig.csv"), index=False)